## MIDI Decoder example

The chunk of code bellow illustrates how to use the MIDI Decoder from the [make_token_sequence_dataset](../../src/data/make_token_sequence_dataset.py) module to decode text to MIDI

In [7]:
# Imports python modules located at the src/data directory (make_token_sequence_dataset)
import sys
sys.path.append('../..')
import note_seq
import json


MIDI_OUTPUT = 'decoded_prelude_in_c_major.mid'

PRIMERS_PATH = f'../../resources/tokens/primers_token_sequences.json'
encoded_primers_options = ['clair_de_lune.mid', 'c_major_arpeggio.mid', 'c_major_scale.mid', 'fur_elise.mid', 'moonlight_sonata.mid', 'prelude_in_c_major.mid']

with open(PRIMERS_PATH, 'r') as file:
    PRIMERS = json.load(file)

encoded_notes = PRIMERS.get('prelude_in_c_major.mid')
print(encoded_notes)

piece_start time_shift=22 velocity=12 note_on=60 time_shift=29 note_on=64 time_shift=23 velocity=15 note_on=67 time_shift=22 note_on=72 time_shift=22 velocity=16 note_on=76 time_shift=23 note_off=67 velocity=15 note_on=67 time_shift=22 note_off=72 note_on=72 time_shift=23 note_off=76 note_on=76 time_shift=19 note_off=60 note_on=60 time_shift=22 note_off=64 note_on=64 time_shift=23 note_off=67 note_on=67 time_shift=19 note_off=72 velocity=16 note_on=72 time_shift=22 note_off=76 velocity=15 note_on=76 time_shift=19 note_off=67 velocity=16 note_on=67 time_shift=20 note_off=72 velocity=17 note_on=72 time_shift=22 note_off=76 velocity=16 note_on=76 time_shift=18 note_off=60 note_off=64 note_off=67 note_off=72 note_off=76 piece_end


Removes piece_start and piece_end tokens, since they are artificially created

In [2]:
valid_encoded_notes = [token for token in  encoded_notes.split(" ") if '=' in token]
valid_encoded_notes

['time_shift=22',
 'velocity=12',
 'note_on=60',
 'time_shift=29',
 'note_on=64',
 'time_shift=23',
 'velocity=15',
 'note_on=67',
 'time_shift=22',
 'note_on=72',
 'time_shift=22',
 'velocity=16',
 'note_on=76',
 'time_shift=23',
 'note_off=67',
 'velocity=15',
 'note_on=67',
 'time_shift=22',
 'note_off=72',
 'note_on=72',
 'time_shift=23',
 'note_off=76',
 'note_on=76',
 'time_shift=19',
 'note_off=60',
 'note_on=60',
 'time_shift=22',
 'note_off=64',
 'note_on=64',
 'time_shift=23',
 'note_off=67',
 'note_on=67',
 'time_shift=19',
 'note_off=72',
 'velocity=16',
 'note_on=72',
 'time_shift=22',
 'note_off=76',
 'velocity=15',
 'note_on=76',
 'time_shift=19',
 'note_off=67',
 'velocity=16',
 'note_on=67',
 'time_shift=20',
 'note_off=72',
 'velocity=17',
 'note_on=72',
 'time_shift=22',
 'note_off=76',
 'velocity=16',
 'note_on=76',
 'time_shift=18',
 'note_off=60',
 'note_off=64',
 'note_off=67',
 'note_off=72',
 'note_off=76']

# Extract performance events from encoded tokens

Extract performance event from token value

In [3]:
from note_seq.performance_lib import PerformanceEvent

def extract_performance_from_token(token):
    event_type, str_value = token.split('=')
    value = int(str_value)

    if event_type == "note_on":
        return PerformanceEvent(event_type=PerformanceEvent.NOTE_ON, event_value=value)
    elif event_type == "note_off":
        return PerformanceEvent(event_type=PerformanceEvent.NOTE_OFF, event_value=value)
    elif event_type == "time_shift":
        return PerformanceEvent(event_type=PerformanceEvent.TIME_SHIFT, event_value=value)
    elif event_type == "velocity":
        return PerformanceEvent(event_type=PerformanceEvent.VELOCITY, event_value=value)
    else:
        raise Exception("Unknown event type: " + event_type)

performance_event_list = []

for token in valid_encoded_notes:
    performance_event_list.append(extract_performance_from_token(token))

# Define functions for reverse engineering

Define necessary functions to do reverse engineering in order to transform the performance event list to a note_seq note sequence

In [4]:
import collections
import math
from note_seq.protobuf import music_pb2
from note_seq import constants

MAX_MIDI_VELOCITY = 127
VELOCITY_BINS = 32
BIN_SIZE = int(math.ceil(MAX_MIDI_VELOCITY / VELOCITY_BINS))  # Ceil(127 / 32) = 4

STEPS_PER_SECOND = 100
SECONDS_PER_STEP = 1.0 / STEPS_PER_SECOND

def velocity_bin_to_velocity(velocity_bin):
    return  1 + (velocity_bin - 1) * 4

def add_note_to_sequence(sequence, note_pitch, pitch_start_step, current_step, pitch_velocity):
    note = sequence.notes.add()
    note.start_time = pitch_start_step * SECONDS_PER_STEP
    note.end_time = current_step * SECONDS_PER_STEP

    note.pitch = note_pitch
    note.velocity = pitch_velocity
    note.instrument = 0

    if note.end_time > sequence.total_time:
        sequence.total_time = note.end_time

    return sequence

def extract_sequence_from_performance_events(performance_event_list):
    current_step = 0
    velocity = 0
    pitch_start_steps_and_velocities = collections.defaultdict(list)

    sequence = music_pb2.NoteSequence()
    sequence.ticks_per_quarter = constants.STANDARD_PPQ

    for event in performance_event_list:
        if event.event_type == PerformanceEvent.TIME_SHIFT:
            current_step += event.event_value

        elif event.event_type == PerformanceEvent.VELOCITY:
            velocity = velocity_bin_to_velocity(event.event_value)

        elif event.event_type == PerformanceEvent.NOTE_ON:
            pitch_start_steps_and_velocities[event.event_value].append((current_step, velocity))

        elif event.event_type == PerformanceEvent.NOTE_OFF:
            pitch_start_step, pitch_velocity = pitch_start_steps_and_velocities[event.event_value][0]

            # Removes the retrieved elements (pitch_start_step, pitch_velocity) from the list
            pitch_start_steps_and_velocities[event.event_value] = (
                  pitch_start_steps_and_velocities[event.event_value][1:])

            if current_step == pitch_start_step:
                print('Ignoring note with zero duration at step ' + current_step)
                continue

            sequence = add_note_to_sequence(
                sequence,
                event.event_value,
                pitch_start_step,
                current_step,
                pitch_velocity)
        else:
            raise Exception("Unknown event type: " + event.event_type)

    # There could be remaining pitches that were never ended. End them now and create notes.
    for pitch in pitch_start_steps_and_velocities:
      for pitch_start_step, pitch_velocity in pitch_start_steps_and_velocities[pitch]:

        if current_step == pitch_start_step:
            print('Ignoring note with zero duration at step ' + current_step)
            continue

        sequence = add_note_to_sequence(
            sequence,
            event.event_value,
            pitch_start_step,
            current_step,
            pitch_velocity)

    return sequence

# Extract note sequence from performance events

Apply `extract_sequence_from_performance_events` function to previously defined `performance_event_list` to generate a new MIDI file

In [8]:
sequence = extract_sequence_from_performance_events(performance_event_list)
note_seq.midi_io.note_sequence_to_midi_file(sequence, MIDI_OUTPUT)

# Play and plot of MIDI Extract

In [6]:
note_seq.play_sequence(sequence)
note_seq.plot_sequence(sequence)

Loading BokehJS ...